In [24]:
# imports
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


In [25]:
# constants

#MODEL_GPT = 'gpt-4o-mini'

# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"


In [26]:
# set up environment

system_prompt = "You are an expert code tutor with a knack for explaining code in a structured and beginner-friendly way. You will be sent snippets of code, and your job is to:\n\n"
system_prompt += "1. Clearly explain what the code does in detail, step by step.\n"
system_prompt += "2. Provide a structured explanation using markdown formatting.\n"
system_prompt += "3. Use headings, bullet points, and code blocks to make the explanation engaging and easy to follow.\n"
system_prompt += "4. Summarize the code's purpose concisely at the end.\n\n"
system_prompt += "Here are examples of how you should respond:\n\n"
system_prompt += "---\n\n"
system_prompt += "**Example 1: Python Function**\n\n"
system_prompt += "Input Code:\n"
system_prompt += "```python\n"
system_prompt += "def add_numbers(a, b):\n"
system_prompt += "    return a + b\n"
system_prompt += "```\n\n"
system_prompt += "Response:\n"
system_prompt += "```markdown\n"
system_prompt += "### Explanation\n\n"
system_prompt += "**Code Overview**:\n"
system_prompt += "This code defines a simple Python function that takes two arguments and returns their sum.\n\n"
system_prompt += "**Breakdown**:\n"
system_prompt += "1. `def add_numbers(a, b):`:\n"
system_prompt += "   - Defines a function named `add_numbers` with two parameters, `a` and `b`.\n"
system_prompt += "2. `return a + b`:\n"
system_prompt += "   - Computes the sum of `a` and `b` and returns the result.\n\n"
system_prompt += "**Purpose**:\n"
system_prompt += "This function provides a reusable way to calculate the sum of two numbers.\n"
system_prompt += "```\n\n"
system_prompt += "---\n\n"
system_prompt += "**Example 2: Looping Through a List**\n\n"
system_prompt += "Input Code:\n"
system_prompt += "```python\n"
system_prompt += "fruits = [\"apple\", \"banana\", \"cherry\"]\n"
system_prompt += "for fruit in fruits:\n"
system_prompt += "    print(fruit)\n"
system_prompt += "```\n\n"
system_prompt += "Response:\n"
system_prompt += "```markdown\n"
system_prompt += "### Explanation\n\n"
system_prompt += "**Code Overview**:\n"
system_prompt += "This code iterates through a list of fruits and prints each fruit to the console.\n\n"
system_prompt += "**Breakdown**:\n"
system_prompt += "1. `fruits = [\"apple\", \"banana\", \"cherry\"]`:\n"
system_prompt += "   - Initializes a list containing three strings: `\"apple\"`, `\"banana\"`, and `\"cherry\"`.\n"
system_prompt += "2. `for fruit in fruits:`:\n"
system_prompt += "   - Starts a loop that iterates over each element in the `fruits` list.\n"
system_prompt += "3. `print(fruit)`:\n"
system_prompt += "   - Prints the current element (fruit) in the loop to the console.\n\n"
system_prompt += "**Purpose**:\n"
system_prompt += "This code demonstrates how to iterate through a list and perform an action for each element.\n"
system_prompt += "```\n\n"
system_prompt += "---\n\n"
system_prompt += "Follow this structure and ensure clarity in your explanations. If code is incorrect or could be optimized, provide suggestions in a helpful tone.\n"


In [27]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [28]:
user_prompt = question

In [29]:
def messages_for_code():
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [30]:
def explain_code():
     payload = {
        "model": MODEL,
        "messages": messages_for_code(),
        "stream":  False
    }
     response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
     return (response.json()['message']['content'])

In [31]:
explain_code()

'### Explanation\n\n#### Code Overview\nThis code uses a combination of Python\'s dictionary and generator features to extract author names from a list of books.\n\n#### Breakdown\n1. `yield from`:\n   - This is a syntax feature introduced in Python 3.3 that allows the use of the `yield from` expression, which delegates the iteration over the object inside it.\n2. `{book.get("author") for book in books if book.get("author")}`:\n   - This is a generator expression, also known as a list comprehension with parentheses instead of curly brackets.\n   - It iterates over each book in the `books` collection (assuming it\'s a dictionary or iterable that contains dictionaries).\n   - The `if` condition filters out books without an "author" key.\n   - For each book, it extracts and yields the value associated with the "author" key.\n\n#### How it Works\nThe code works as follows:\n- It iterates over each book in the `books` collection.\n- For each book, it checks if the "author" key exists. If it

In [32]:
# Get gpt-4o-mini to answer, with streaming

In [33]:
# Get Llama 3.2 to answer

def explain_code_stream():
    payload = {
        "model": MODEL,
        "messages": messages_for_code(),
        "stream":  True
    }

     # Initialize the display for live updates
    display_handle = display(Markdown(""), display_id=True)
    response_accumulated = ""  # Accumulated response

    try:
        with requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True) as response:
            response.raise_for_status()  # Ensure the request was successful
            
            # Iterate through streamed response lines
            for line in response.iter_lines():
                if line:  # Ignore keep-alive newlines
                    try:
                        data = json.loads(line)
                        if "message" in data and "content" in data["message"]:
                            content = data["message"]["content"]
                            response_accumulated += content
                            # Clean and update the markdown display
                            formatted_response = response_accumulated.replace("```", "").replace("markdown", "")
                            display_handle.update(Markdown(formatted_response))  # Update the display
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Error with the request: {e}")

In [34]:
explain_code_stream()

### Explanation

**Code Overview**
This code uses a combination of dictionary methods and generator expressions to extract author names from a list of books.

**Breakdown**

* `yield from`: This keyword is used to delegate the iteration over the yielded values to another iterable. In this case, it delegates the iteration over the results of the expression that follows.
* `{...}`: This is a dictionary literal comprehension, which creates a new dictionary containing key-value pairs generated by evaluating the expression inside.
* `book.get("author")`: For each book in the list, this line attempts to retrieve the value associated with the "author" key. If the key does not exist, `get` returns `None`.
* `{... for book in books if ...}`: This is a generator expression that iterates over the list of books using the loop variable `book`. The `if` clause filters out any book dictionaries that do not contain an "author" key.

**Purpose**
This code generates an iterable sequence of author names from a list of book dictionaries, filtering out any books without authors. The resulting iterable can be used in various contexts, such as:

* Iterating over the author names to display them in a UI
* Using as input for another function that expects an iterable of strings
* Assigning to a variable or using as a key-value pair in another data structure

### Example Use Case

Suppose you have a list of book dictionaries like this:
python
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "publisher": "ABC Inc."},
    {"title": "Book 3", "author": "Jane Smith"}
]

Using the code snippet provided, you can get an iterable sequence of author names like this:
python
authors = yield from {book.get("author") for book in books if book.get("author")}
print(authors)  # Output: ["John Doe", "Jane Smith"]

Note that `yield from` is a shorthand way to implement iterators, making the code more concise and expressive.